In [ ]:
import scqubits as scq
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [ ]:
qub_name = "S001"

loadpath = f"../result/{qub_name}/params.csv"
df = pd.read_csv(loadpath)
params = df.loc[0, ["EJ", "EC", "EL"]]
cflx, period = df.loc[0, ["Half flux", "Period"]]


def phi2mA(ratio):
    return (period * (ratio - 0.5) + cflx) * 1e3


df.head()

In [ ]:
r_f = 7.48675
g = 0.034

In [ ]:
flxs = np.linspace(0, 0.51, 1001)
mAs = phi2mA(flxs)

fluxonium = scq.Fluxonium(
    *params, flux=0.5, cutoff=40, truncated_dim=15, id_str="qubit"
)
resonator = scq.Oscillator(r_f, truncated_dim=10, id_str="resonator")
hilbertspace = scq.HilbertSpace([fluxonium, resonator])
hilbertspace.add_interaction(
    g=g,
    op1=fluxonium.n_operator,
    op2=resonator.creation_operator,
    add_hc=True,
    id_str="q-r coupling",
)


def update_hilbertspace(flux):
    fluxonium.flux = flux


sweep = scq.ParameterSweep(
    hilbertspace,
    {"flxs": flxs},
    update_hilbertspace=update_hilbertspace,
    evals_count=20,
    subsys_update_info={"flxs": [fluxonium]},
)

In [ ]:
predict_chi = sweep["chi"]["subsys1":0, "subsys2":1].toarray()
dispersives_01 = predict_chi[:, 1] * 1e3
dispersives_02 = predict_chi[:, 2] * 1e3

In [ ]:
fig = go.Figure()
fig.add_scatter(
    x=flxs,
    y=dispersives_01,
    mode="lines",
    line=dict(color="blue"),
    name=r"$\chi_{01}$",
)
fig.add_scatter(
    x=flxs,
    y=dispersives_02,
    mode="lines",
    line=dict(color="red"),
    name=r"$\chi_{02}$",
)

# plot a dash hline to indicate the 0 point, also add a xaxis2 to show mA
fig.add_scatter(
    x=mAs, y=np.zeros_like(mAs), xaxis="x2", line=dict(color="black", dash="dash")
)
mAs_ticks = mAs[:: len(mAs) // 10]
fig.update_layout(
    xaxis2=dict(
        tickvals=mAs_ticks,
        ticktext=[f"{mA:.2f}" for mA in mAs_ticks],
        overlaying="x",
        side="top",
        title_text="mA",
    )
)

fig.update_layout(
    xaxis_title=r"$Φ_{ext}/Φ_0$",
    yaxis_title=r"$\chi (MHz)$",
    legend_title="Dispersive",
    margin=dict(l=0, r=0, t=0, b=0),
)
fig.update_yaxes(range=[-2, 2])

fig.show()

In [ ]:
fig.write_html(f"../result/{qub_name}/web/dispersive.html", include_plotlyjs="cdn")
fig.write_image(
    f"../result/{qub_name}/image/dispersive.png", format="png", width=800, height=400
)